# Lab 7 - Logistic Regression, ROCs and imputation


In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.api import OLS
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
%matplotlib inline
import seaborn.apionly as sns

## Asymmetry

First we'd like to simply notice that our data set is very highly asymmetric, with positives, or people who churned, only making up 14-15% of the samples.

In [ ]:
gssdata=pd.read_csv("gssdata4.csv")
gssdata.head()

In [ ]:
poorhealth = np.where(gssdata['health'] == 'poor',1,0)
excellenthealth = np.where(gssdata['health'] == 'excellent',1,0)
fairhealth = np.where(gssdata['health'] == 'fair',1,0)
gssdata['poorhealth'] = poorhealth
gssdata['fairhealth'] = fairhealth
gssdata['excellenthealth'] = excellenthealth
100*gssdata.poorhealth.mean()

 This means that a classifier which predicts EVERY respondent to not be in excellent health will have an accuracy rate of 80-81%. 

But is accuracy the correct metric?

## Fit a logistic model ignoring missingness

Let's begin by fitting a logistic regression model to predict poor health based on all the other predictors in the model.  Be sure to regularize (with CV) to make sure we are not overfitting to the data.

First we need to do a small amount of data clean-up (ignoring missingness for now in income).

In [ ]:
#creating dummies two ways
gssdata['female'] = 1*(gssdata['sex'] ==  'female')
dummy_vars = pd.get_dummies(gssdata[['sexornt','partyid','race']])
gssdata = gssdata.join(dummy_vars)

In [ ]:
print(gssdata.shape)
gssdata.head()

In [ ]:
gssdata_full = gssdata.dropna(how = 'any')
print(gssdata_full.shape)
gssdata_full.head()

In [ ]:
from sklearn.model_selection import train_test_split
itrain, itest = train_test_split(range(gssdata_full.shape[0]), train_size=0.50)
#gsstemp = gssdata_full.drop(['health','fairhealth','goodhealth','excellenthealth','sex','sexornt','partyid','race'],axis=1)
gsstemp = gssdata_full[['age','educ','female','partyid_dem','partyid_rep','income']]
X_train = gsstemp.iloc[itrain, :]
X_test = gsstemp.iloc[itest, :]
y_train = gssdata_full['poorhealth'].iloc[itrain]
y_test = gssdata_full['poorhealth'].iloc[itest]

y_train.shape, X_train.shape, y_test.shape, X_test.shape



In [ ]:
gsstemp.head()
100*gssdata_full.poorhealth.mean()

In [ ]:
logit = LogisticRegression(C=1000000)
logit.fit(X_train, y_train) 
print(logit.score(X_test,y_test))
logit.coef_

## Remember the Confusion matrix? We reproduce it here for convenience

- the samples that are +ive and the classifier predicts as +ive are called True Positives (TP)
- the samples that are -ive and the classifier predicts (wrongly) as +ive are called False Positives (FP)
- the samples that are -ive and the classifier predicts as -ive are called True Negatives (TN)
- the samples that are +ive and the classifier predicts as -ive are called False Negatives (FN)

A classifier produces a confusion matrix which looks like this:

![confusionmatrix](./confusionmatrix_360.png)


IMPORTANT NOTE: In sklearn, to obtain the confusion matrix in the form above, always have the observed `y` first, i.e.: use as `confusion_matrix(y_true, y_pred)`



In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,logit.predict(X_test)))
yhats = logit.predict_proba(X_train)
plt.hist(yhats[:,1])
#print(confusion_matrix(y_test,))


In [ ]:
#manually making confusion table from a different threshold
def t_repredict(est, t, xtest):
    probs = est.predict_proba(xtest)
    p0 = probs[:,0]
    p1 = probs[:,1]
    ypred = (p1 > t)*1
    return ypred

#your code here: look at other thresholds for confusion matrix

In [ ]:
#making ROC curves for this model
from sklearn.metrics import roc_curve, auc

def make_roc(name, clf, ytest, xtest, ax=None, labe=5, proba=True, skip=0):
    initial=False
    if not ax:
        ax=plt.gca()
        initial=True
    if proba:#for stuff like logistic regression
        fpr, tpr, thresholds=roc_curve(ytest, clf.predict_proba(xtest)[:,1])
    else:#for stuff like SVM
        fpr, tpr, thresholds=roc_curve(ytest, clf.decision_function(xtest))
    roc_auc = auc(fpr, tpr)
    if skip:
        l=fpr.shape[0]
        ax.plot(fpr[0:l:skip], tpr[0:l:skip], '.-', alpha=0.3, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    else:
        ax.plot(fpr, tpr, '.-', alpha=0.3, label='ROC curve for %s (area = %0.2f)' % (name, roc_auc))
    label_kwargs = {}
    label_kwargs['bbox'] = dict(
        boxstyle='round,pad=0.3', alpha=0.2,
    )
    if labe!=None:
        for k in range(0, fpr.shape[0],labe):
            #from https://gist.github.com/podshumok/c1d1c9394335d86255b8
            threshold = str(np.round(thresholds[k], 2))
            ax.annotate(threshold, (fpr[k], tpr[k]), **label_kwargs)
    if initial:
        ax.plot([0, 1], [0, 1], 'k--')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title('ROC')
    ax.legend(loc="lower right")
    return ax

sns.set_context("poster")
ax=make_roc("logistic",logit, y_test, X_test, labe=10, skip=5)


## Let's get back the data with missingness

In [ ]:
#first build a model to impute using data without missing 
plt.hist(gssdata_full['income'])

In [ ]:
#back to the data set with missingness and impute the mean
gssdata2 = gssdata.copy()
gssdata2['income'] = gssdata['income'].fillna(gssdata_full['income'].mean())
plt.hist(gssdata2['income'])

In [ ]:
from sklearn.model_selection import train_test_split
itrain, itest = train_test_split(range(gssdata2.shape[0]), train_size=0.50)
#gsstemp = gssdata_full.drop(['health','fairhealth','goodhealth','excellenthealth','sex','sexornt','partyid','race'],axis=1)
gsstemp = gssdata2[['age','educ','female','partyid_dem','partyid_rep','income']]
X_train2 = gsstemp.iloc[itrain, :]
X_test2 = gsstemp.iloc[itest, :]
y_train2 = gssdata2['poorhealth'].iloc[itrain]
y_test2 = gssdata2['poorhealth'].iloc[itest]

y_train.shape, X_train.shape, y_test.shape, X_test.shape

logit2 = LogisticRegression(C=1000000)
logit2.fit(X_train2, y_train2) 
print(logit2.score(X_test2,y_test2))
logit2.coef_

In [ ]:
#your code here: create confusion table

In [ ]:
#your code here: create an ROC curve

## Imputation with a model

In [ ]:
#use the dataset without NAs to build a model
from sklearn.metrics import mean_squared_error

gssdata_full.head()

X_imp = gssdata_full[['age','educ','female','partyid_dem','partyid_rep']]
y_imp = gssdata_full['income']

from sklearn.linear_model import LinearRegression

regress = LinearRegression()
regress.fit(X_imp,y_imp)
y_hat = regress.predict(X_imp)

#your code here: do the imputation

In [ ]:
missing_index = gssdata.income[gssdata.income.isnull()].index
missing_series = pd.Series(data = y_missing_noise, index = missing_index)

In [ ]:
#back to the data set with missingness and impute the predictions
gssdata_imp = gssdata.copy()

gssdata_imp['income'] = gssdata_imp['income'].fillna(missing_series)

In [ ]:
plt.hist(gssdata_imp['income'])

In [ ]:
gsstemp = gssdata_imp[['age','educ','female','partyid_dem','partyid_rep','income']]
X_train3 = gsstemp.iloc[itrain, :]
X_test3 = gsstemp.iloc[itest, :]
#y_train3 = gssdata_imp['poorhealth'].iloc[itrain]
#y_test3 = gssdata_imp['poorhealth'].iloc[itest]

y_train.shape, X_train.shape, y_test.shape, X_test.shape

logit3 = LogisticRegression(C=1000000)
logit3.fit(X_train3, y_train2) 
print(logit3.score(X_test3,y_test2))
logit3.coef_

In [ ]:
print(confusion_matrix(y_test2,t_repredict(logit3, 0.5, X_test3)))
print(confusion_matrix(y_train2,t_repredict(logit3, 0.5, X_train3)))

print(confusion_matrix(y_test2,t_repredict(logit3, 0.1, X_test3)))
print(confusion_matrix(y_train2,t_repredict(logit3, 0.1, X_train3)))

In [ ]:
sns.set_context("poster")
ax=make_roc("logistic",logit3, y_test2, X_test3, labe=10, skip=5)